In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/housedata/output.csv
/kaggle/input/housedata/data.csv
/kaggle/input/housedata/data.dat


In [2]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
h_dat = pd.read_csv('../input/housedata/data.csv')
print(h_dat.shape)
print('')
h_dat.head()

(4600, 18)



,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2014-05-02 00:00:00,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2014-05-02 00:00:00,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2014-05-02 00:00:00,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2014-05-02 00:00:00,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [4]:
h_dat.columns

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'street', 'city',
       'statezip', 'country'],
      dtype='object')

In [5]:
df = h_dat.set_index('country')
df

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip
country,,,,,,,,,,,,,,,,,
USA,2014-05-02 00:00:00,3.130000e+05,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133
USA,2014-05-02 00:00:00,2.384000e+06,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119
USA,2014-05-02 00:00:00,3.420000e+05,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042
USA,2014-05-02 00:00:00,4.200000e+05,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008
USA,2014-05-02 00:00:00,5.500000e+05,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USA,2014-07-09 00:00:00,3.081667e+05,3.0,1.75,1510,6360,1.0,0,0,4,1510,0,1954,1979,501 N 143rd St,Seattle,WA 98133
USA,2014-07-09 00:00:00,5.343333e+05,3.0,2.50,1460,7573,2.0,0,0,3,1460,0,1983,2009,14855 SE 10th Pl,Bellevue,WA 98007
USA,2014-07-09 00:00:00,4.169042e+05,3.0,2.50,3010,7014,2.0,0,0,3,3010,0,2009,0,759 Ilwaco Pl NE,Renton,WA 98059


# **Empty/NULL values**

Testing the empty columns

In [6]:
print(h_dat.shape)

house_missing_vals = (df.isnull().sum())
print(house_missing_vals[house_missing_vals > 0])

# Since no Empty Values we dont need to do anything.

(4600, 18)
Series([], dtype: int64)


In [7]:
h_dat.sort_values(by='bedrooms', ascending=False)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
241,2014-05-07 00:00:00,599999.0,9.0,4.50,3830,6988,2.5,0,0,3,2450,1380,1938,2003,8809 Densmore Ave N,Seattle,WA 98103,USA
3911,2014-07-03 00:00:00,340000.0,8.0,2.75,2790,6695,1.0,0,0,3,1470,1320,1977,2004,17512 Corliss Ave N,Shoreline,WA 98133,USA
1500,2014-05-28 00:00:00,1970000.0,8.0,3.50,4440,6480,2.0,0,3,5,3140,1300,1959,0,1210 22nd Ave E,Seattle,WA 98112,USA
1644,2014-05-30 00:00:00,840000.0,7.0,4.50,4290,37607,1.5,0,0,5,4290,0,1982,0,13527 231st Pl SE,Issaquah,WA 98027,USA
291,2014-05-07 00:00:00,540000.0,7.0,5.75,3700,7647,2.0,0,1,3,3700,0,1948,1984,15007-15299 37th Ave NE,Lake Forest Park,WA 98155,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,2014-05-22 00:00:00,353000.0,1.0,1.00,550,1279,2.0,0,0,3,550,0,2008,0,1620 14th Ave,Seattle,WA 98122,USA
1906,2014-06-04 00:00:00,129000.0,1.0,1.00,650,15364,1.0,0,0,4,650,0,1967,0,13712 SE 256th Pl,Kent,WA 98042,USA
260,2014-05-07 00:00:00,250000.0,1.0,1.00,1100,4373,1.0,0,0,2,820,280,1947,0,232 SW 154th St,Burien,WA 98166,USA
2365,2014-06-12 00:00:00,1095000.0,0.0,0.00,3064,4764,3.5,0,2,3,3064,0,1990,2009,814 E Howe St,Seattle,WA 98102,USA


In [8]:
h_dat.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
count,4.600000e+03,4600.000000,4600.000000,4600.000000,4.600000e+03,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000
mean,5.519630e+05,3.400870,2.160815,2139.346957,1.485252e+04,1.512065,0.007174,0.240652,3.451739,1827.265435,312.081522,1970.786304,808.608261
std,5.638347e+05,0.908848,0.783781,963.206916,3.588444e+04,0.538288,0.084404,0.778405,0.677230,862.168977,464.137228,29.731848,979.414536
min,0.000000e+00,0.000000,0.000000,370.000000,6.380000e+02,1.000000,0.000000,0.000000,1.000000,370.000000,0.000000,1900.000000,0.000000
25%,3.228750e+05,3.000000,1.750000,1460.000000,5.000750e+03,1.000000,0.000000,0.000000,3.000000,1190.000000,0.000000,1951.000000,0.000000
50%,4.609435e+05,3.000000,2.250000,1980.000000,7.683000e+03,1.500000,0.000000,0.000000,3.000000,1590.000000,0.000000,1976.000000,0.000000
75%,6.549625e+05,4.000000,2.500000,2620.000000,1.100125e+04,2.000000,0.000000,0.000000,4.000000,2300.000000,610.000000,1997.000000,1999.000000
max,2.659000e+07,9.000000,8.000000,13540.000000,1.074218e+06,3.500000,1.000000,4.000000,5.000000,9410.000000,4820.000000,2014.000000,2014.000000


# **Phase 2**

* Selecting the Traget Feature.
* Choosing the important features.

In [9]:
y = h_dat.price
h_dat_features = ['bedrooms', 'bathrooms', 'sqft_living', 'floors', 'condition', 'yr_built']
X = h_dat[h_dat_features]

In [10]:
h_dat.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
count,4.600000e+03,4600.000000,4600.000000,4600.000000,4.600000e+03,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000
mean,5.519630e+05,3.400870,2.160815,2139.346957,1.485252e+04,1.512065,0.007174,0.240652,3.451739,1827.265435,312.081522,1970.786304,808.608261
std,5.638347e+05,0.908848,0.783781,963.206916,3.588444e+04,0.538288,0.084404,0.778405,0.677230,862.168977,464.137228,29.731848,979.414536
min,0.000000e+00,0.000000,0.000000,370.000000,6.380000e+02,1.000000,0.000000,0.000000,1.000000,370.000000,0.000000,1900.000000,0.000000
25%,3.228750e+05,3.000000,1.750000,1460.000000,5.000750e+03,1.000000,0.000000,0.000000,3.000000,1190.000000,0.000000,1951.000000,0.000000
50%,4.609435e+05,3.000000,2.250000,1980.000000,7.683000e+03,1.500000,0.000000,0.000000,3.000000,1590.000000,0.000000,1976.000000,0.000000
75%,6.549625e+05,4.000000,2.500000,2620.000000,1.100125e+04,2.000000,0.000000,0.000000,4.000000,2300.000000,610.000000,1997.000000,1999.000000
max,2.659000e+07,9.000000,8.000000,13540.000000,1.074218e+06,3.500000,1.000000,4.000000,5.000000,9410.000000,4820.000000,2014.000000,2014.000000


# **Importing the Model**

In [11]:
h_dat_model = DecisionTreeRegressor(random_state=1)

h_dat_model.fit(X, y)

DecisionTreeRegressor(random_state=1)

In [12]:
print("Making predictions for the following 5 houses:")
print(X.head())
print("The predictions are")
print(h_dat_model.predict(X.head()))

Making predictions for the following 5 houses:
   bedrooms  bathrooms  sqft_living  floors  condition  yr_built
0       3.0       1.50         1340     1.5          3      1955
1       5.0       2.50         3650     2.0          5      1921
2       3.0       2.00         1930     1.0          4      1966
3       3.0       2.25         2000     1.0          4      1963
4       4.0       2.50         1940     1.0          4      1976
The predictions are
[ 313000. 2384000.  342000.  420000.  550000.]


# **Validating the Model**

In [13]:
h_price_pred = h_dat_model.predict(X)
mean_absolute_error(y , h_price_pred)

1821.2713250517393

In [14]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

h_dat_model = DecisionTreeRegressor()
h_dat_model.fit(train_X, train_y)

h_price_pred = h_dat_model.predict(val_X)
val_mae = mean_absolute_error(val_y , h_price_pred)
print("Validation MAE: {:,.0f}".format(val_mae))

Validation MAE: 273,441


# **Testing for Overfitting and Underfitting**

In [15]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

# **Comparing Tree size values**

In [16]:
cand_max_leaf_nodes = [5, 25, 50, 100, 250, 500]
# Write loop to find the ideal tree size from candidate_max_leaf_nodes
scores = {leaf_size: get_mae(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in cand_max_leaf_nodes}
best_tree_size = min(scores, key=scores.get)

# for cand_max_leaf_nodes in [5, 10, 15, 50, 500, 5000]:
#     my_mae = get_mae(cand_max_leaf_nodes, train_X, val_X, train_y, val_y)
#     print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(cand_max_leaf_nodes, my_mae))

# **Model training and testing**

* Finally we train and fit the model.
* And check for the Mean Absolute Error.

In [17]:
final_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state=1)
final_model.fit(train_X, train_y)

f_price_pred = final_model.predict(val_X)
mae_val = mean_absolute_error(val_y, f_price_pred)
print("Validation MAE: {:,.0f}".format(mae_val))

Validation MAE: 185,239
